In [2]:
!pip install hyperopt
!pip install tqdm
!pip install numpy
!pip install torch
!pip install torch_geometric

In [1]:
import torch
from torch import nn
from torch_geometric.nn import GCNConv, GATConv, ChebConv, SAGEConv
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.utils import add_self_loops
from hyperopt import hp
import numpy as np
from tqdm.notebook import tqdm
from torch_geometric.data import Data

In [2]:
def get_graph(nodes_count, edges_count):
        
    # Import required libraries
    import psycopg2
    import numpy as np
    import csv
    import re
    import time
    from psycopg2 import sql

    # Helper function to connect to SQL database
    def sql_connector(dataset_name):
        connection = psycopg2.connect(
            dbname=db_name,
            user='postgres',
            password='Berlin!321',
            host='localhost'
        )

        return connection

    # Function to extract number of noes and edges from the file name
    def extract_file_info(file_name):
        """
        Extracts the number of nodes and edges from the filename.

        The filename is expected to be in the format `X_<num_nodes>_nodes_<num_edges>_edges.csv`.

        Parameters:
            filename (str): Path to the file with encoded information on nodes and edges.

        Returns:
            tuple: A tuple containing two integers:
                - num_nodes (int): The number of nodes extracted from the filename.
                - num_edges (int): The number of edges extracted from the filename.
        """

        match = re.search(r'(\d+)_nodes_(\d+)_edges', file_name)
        if match:
            num_nodes = int(match.group(1))
            num_edges = int(match.group(2))
            return num_nodes, num_edges
        else:
            raise ValueError(f"Filename format is not recognized: {file_name}")

    # Function to create a database
    def create_database(db_name):
        """
        Creates a new PostgreSQL database.

        Parameters:
        db_name (str): The name of the database to be created.
        """

        connection = psycopg2.connect(
            dbname='postgres',  # Connect to default db to create new db
            user='postgres',
            password='Berlin!321',
            host='localhost'
        )
        connection.autocommit = True

        cursor = connection.cursor()

        try:
            # Create the database
            cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name)))
            print(f"Database {db_name} created successfully.")
            
            # Connect to the newly created database and enable pg_stat_statements
            new_db_connection = psycopg2.connect(
                dbname=db_name,
                user='postgres',
                password='Berlin!321',
                host='localhost'
            )
            new_db_cursor = new_db_connection.cursor()
            
            new_db_cursor.close()
            new_db_connection.close()

        except psycopg2.Error as e:
            print(f"Error: {e}")
        finally:
            cursor.close()
            connection.close()

    # Function to create tables in the PostgreSQL database with indexing and track index creation times
    def create_tables_with_index(connection):
        """
        Creates the tables 'nodes', 'edges', and 'labels' in the connected PostgreSQL database, if they do not exist.
        Also adds indexes on certain columns to improve query performance.

        Parameters:
            connection (PostgreSQL connection object): Connection to the PostgreSQL database.

        Returns:
            tuple: A tuple containing two values:
                - mean_data_loading_index_time (float): Mean execution time for creating the indexes.
                - std_data_loading_index_time (float): Standard deviation of the execution time for creating the indexes.
        """
        cursor = connection.cursor()

        # Function to execute an index creation and track the time
        def create_index(query):
            cursor.execute(query)     # Execute index creation
            connection.commit()       # Commit after each index creation
        
        # Create the edges table and its indexes
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS edges (
                source INT, 
                target INT
            )
        """)
        
        # Create indexes and track their creation times
        create_index("CREATE INDEX IF NOT EXISTS source_index ON edges (source)")
        create_index("CREATE INDEX IF NOT EXISTS target_index ON edges (target)")
        create_index("CREATE INDEX IF NOT EXISTS source_target_index ON edges (source, target)")
        create_index("CREATE INDEX IF NOT EXISTS target_source_index ON edges (target, source)")

        # Create the nodes table and its features
        cursor.execute("CREATE TABLE IF NOT EXISTS nodes (node_id SERIAL PRIMARY KEY)")
        
        # Dynamically add features if they do not exist
        cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name='nodes'")
        columns = [column[0] for column in cursor.fetchall()]
        
        for i in range(100):  # Assuming 100 features
            feature_name = f"feature_{i}"
            if feature_name not in columns:
                cursor.execute(f"ALTER TABLE nodes ADD COLUMN {feature_name} FLOAT")

        # Create the labels table and its index
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS labels (
                node_id INT REFERENCES nodes(node_id)
            )
        """)
        create_index("CREATE INDEX IF NOT EXISTS node_id_index ON labels (node_id)")

        connection.commit()
        cursor.close()

    # Function to load edges data into PostgreSQL
    def load_edges(connection, file_path):
        """
        Loads edges from a CSV file into the edges table in the database.

        Parameters:
        connection (psycopg2.extensions.connection): The database connection.
        edge_file (str): The path to the CSV file containing edge data.
        """
        print("Edges are loading!")
        cursor = connection.cursor()
        
        # Read all rows from the CSV file
        with open(file_path, 'r') as edges_file:
            edges_reader = csv.reader(edges_file)
            next(edges_reader)  # Skip header

            # Prepare a multi-row insert statement
            args_str = ','.join(cursor.mogrify("(%s,%s)", (int(row[0]), int(row[1]))).decode("utf-8") 
                                            for row in edges_reader)
            query = f"INSERT INTO edges (source, target) VALUES {args_str};"

            # Execute the query
            cursor.execute(query)

        connection.commit()
        cursor.close()
        print("Edge loading completed!")

    # Function to load nodes data into PostgreSQL 
    def load_nodes(connection, file_path):
        """
        Loads nodes from a CSV file into the nodes table in the database.

        Parameters:
        connection (psycopg2.extensions.connection): The database connection.
        node_file (str): The path to the CSV file containing node data.
        """
        print("Nodes are loading!")
        cursor = connection.cursor()
        
        # Read all rows from the CSV file
        with open(file_path, 'r') as nodes_file:
            nodes_reader = csv.reader(nodes_file)
            next(nodes_reader)  # Skip header

            # Prepare a multi-row insert statement
            args_str = ','.join(
                cursor.mogrify("(" + ",".join(["%s"] * 100) + ")", tuple(map(float, row))).decode("utf-8")
                for row in nodes_reader
            )
            query = f"INSERT INTO nodes ({', '.join([f'feature_{i}' for i in range(100)])}) VALUES {args_str};"

            # Execute the query
            cursor.execute(query)

        connection.commit()
        cursor.close()
        print("Node loading completed!")

    # Function to load labels data into PostgreSQL
    def load_labels(connection, file_path):
        """
        Loads labels from a CSV file into the labels table in the database.

        Parameters:
        connection (psycopg2.extensions.connection): The database connection.
        label_file (str): The path to the CSV file containing label data.
        """
        print("Labels are loading!")
        cursor = connection.cursor()

        # Ensure the 'label' column exists
        cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name='labels'")
        existing_columns = set([column[0] for column in cursor.fetchall()])
        if "label" not in existing_columns:
            cursor.execute("ALTER TABLE labels ADD COLUMN label TEXT")
        
        # Read all rows from the CSV file
        with open(file_path, 'r') as labels_file:
            labels_reader = csv.reader(labels_file)
            next(labels_reader)  # Skip header

            # Prepare a multi-row insert statement
            args_str = ','.join(cursor.mogrify("(%s,%s)", (idx, row[0])).decode("utf-8")
                                for idx, row in enumerate(labels_reader, start=1))
            query = f"INSERT INTO labels (node_id, label) VALUES {args_str};"

            # Execute the query
            cursor.execute(query)

        connection.commit()
        cursor.close()
        print("Label loading completed!")
    
    # Function to get the node table using PostgreSQL    
    def get_node_table_from_sql(dataset_name) -> np.array:  # Change the return type to np.array
        """
        Connects to the database, retrieves node features (excluding node_id) from the nodes table, 
        and returns them as a numpy array.
        
        Parameters:
        dataset_name (str): The name of the dataset (i.e., the database name).
        
        Returns:
        numpy.ndarray: A 2D numpy array of node features.
        """
        # Connect to the PostgreSQL database
        connector = sql_connector(dataset_name)
        
        # Create a cursor from the connection
        cursor = connector.cursor()
        
        # Execute the SQL query
        cursor.execute("SELECT * FROM nodes")  # Select all columns
        node_data = cursor.fetchall()  # Fetch all the rows returned by the query
        
        # Close the cursor
        cursor.close()

        # Convert the data to a numpy array
        node_data = np.array(node_data)

        # Debugging: Print the shape of the node_data
        print("Fetched node data shape:", node_data.shape)

        # Ensure the node data contains the correct number of feature columns
        if node_data.size == 0:  # Check if no data is returned
            raise ValueError("No data returned from the nodes table.")

        if node_data.shape[1] != 101:  # 100 features + 1 node_id column
            raise ValueError(f"Unexpected data shape. Ensure node table contains exactly 100 feature columns plus the node_id. Current shape: {node_data.shape}.")

        # Extract features excluding the node_id column
        node_features = node_data[:, 1:]  # This gets all columns except the first one (node_id)

        return node_features  # Return only the features


    # Function to get the edge index table using PostgreSQL
    def get_edge_table_from_sql(connection):
        """
        Retrieves the edge table from the database.

        Parameters:
        connection (psycopg2.extensions.connection): The database connection.

        Returns:
        list: List of edges as tuples.
        """
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM edges")
        edge_data = cursor.fetchall()

        if not edge_data:
            raise ValueError("No data returned from the edges table.")

        return edge_data

    #  Main block
    if __name__ == "__main__":
        """
        Main execution block of the script.

        This block initializes the parameters, creates the database and tables,
        loads data from CSV files, measures performance for various operations,
        and logs the results.

        The following operations are performed:
        - Loading edges, nodes, and labels.
        - Reading data with a specified number of hops.
        - Updating node features and edge weights.
        - Deleting all data.
        """


        # Define file paths using
        node_file = f'E:/Master Thesis/Thesis_Code/data/X_{nodes_count}_nodes_{edges_count}_edges.csv'
        edge_file = f'E:/Master Thesis/Thesis_Code/data/edge_index_{nodes_count}_nodes_{edges_count}_edges.csv'
        label_file = f'E:/Master Thesis/Thesis_Code/data/y_{nodes_count}_nodes_{edges_count}_edges.csv'
  

        nodes_count, edges_count = extract_file_info(node_file)
        db_name = f"db_{nodes_count}_nodes_{edges_count}_edges"

        # Create a new database
        create_database(db_name)    

        # Connect to the newly created database
        connection = psycopg2.connect(
            dbname=db_name,
            user='postgres',
            password='Berlin!321',
            host='localhost'
        )
        
        # Create tables with indexing
        create_tables_with_index(connection)

        # Load nodes, edges, and labels
        load_nodes(connection, node_file)
        load_edges(connection, edge_file)
        load_labels(connection, label_file)
    
        # Get node features and edge table
        node_features = get_node_table_from_sql(connection)
        print("Node Features Shape:", node_features.shape)

        # Load the labels from the labels table if needed
        cursor = connection.cursor()
        cursor.execute("SELECT label FROM labels")
        labels_data = cursor.fetchall()

        # Extract labels from fetched data
        # node_labels = np.array([label[0] for label in labels_data])  # Assuming label is in the first column
        node_labels = np.array([int(label[0]) for label in labels_data])  # Use float() if needed
        print("Node Labels Shape:", node_labels.shape)

        edge_data = get_edge_table_from_sql(connection)
        edge_index = np.array(edge_data, dtype=np.int64).T  # Ensure edge_index is of type int64
        print("Edge Data Length:", len(edge_index))

        # Close the connection
        connection.close()

        
        return node_features, node_labels, edge_index

In [3]:
def get_graph_batches(nodes_count, edges_count):        
    # Import required libraries
    import psycopg2
    import numpy as np
    import csv
    import re
    import time
    from psycopg2 import sql

    # Helper function to connect to SQL database
    def sql_connector(dataset_name):
        connection = psycopg2.connect(
            dbname=db_name,
            user='postgres',
            password='Berlin!321',
            host='localhost'
        )

        return connection

    # Function to extract number of noes and edges from the file name
    def extract_file_info(file_name):
        """
        Extracts the number of nodes and edges from the filename.

        The filename is expected to be in the format `X_<num_nodes>_nodes_<num_edges>_edges.csv`.

        Parameters:
            filename (str): Path to the file with encoded information on nodes and edges.

        Returns:
            tuple: A tuple containing two integers:
                - num_nodes (int): The number of nodes extracted from the filename.
                - num_edges (int): The number of edges extracted from the filename.
        """

        match = re.search(r'(\d+)_nodes_(\d+)_edges', file_name)
        if match:
            num_nodes = int(match.group(1))
            num_edges = int(match.group(2))
            return num_nodes, num_edges
        else:
            raise ValueError(f"Filename format is not recognized: {file_name}")

    # Function to create a database
    def create_database(db_name):
        """
        Creates a new PostgreSQL database.

        Parameters:
        db_name (str): The name of the database to be created.
        """

        connection = psycopg2.connect(
            dbname='postgres',  # Connect to default db to create new db
            user='postgres',
            password='Berlin!321',
            host='localhost'
        )
        connection.autocommit = True

        cursor = connection.cursor()

        try:
            # Create the database
            cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name)))
            print(f"Database {db_name} created successfully.")
            
            # Connect to the newly created database and enable pg_stat_statements
            new_db_connection = psycopg2.connect(
                dbname=db_name,
                user='postgres',
                password='Berlin!321',
                host='localhost'
            )
            new_db_cursor = new_db_connection.cursor()
            
            new_db_cursor.close()
            new_db_connection.close()

        except psycopg2.Error as e:
            print(f"Error: {e}")
        finally:
            cursor.close()
            connection.close()

    # Function to create tables in the PostgreSQL database with indexing and track index creation times
    def create_tables_with_index(connection):
        """
        Creates the tables 'nodes', 'edges', and 'labels' in the connected PostgreSQL database, if they do not exist.
        Also adds indexes on certain columns to improve query performance.

        Parameters:
            connection (PostgreSQL connection object): Connection to the PostgreSQL database.

        Returns:
            tuple: A tuple containing two values:
                - mean_data_loading_index_time (float): Mean execution time for creating the indexes.
                - std_data_loading_index_time (float): Standard deviation of the execution time for creating the indexes.
        """
        cursor = connection.cursor()

        # Function to execute an index creation and track the time
        def create_index(query):
            cursor.execute(query)     # Execute index creation
            connection.commit()       # Commit after each index creation
        
        # Create the edges table and its indexes
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS edges (
                source INT, 
                target INT
            )
        """)
        
        # Create indexes and track their creation times
        create_index("CREATE INDEX IF NOT EXISTS source_index ON edges (source)")
        create_index("CREATE INDEX IF NOT EXISTS target_index ON edges (target)")
        create_index("CREATE INDEX IF NOT EXISTS source_target_index ON edges (source, target)")
        create_index("CREATE INDEX IF NOT EXISTS target_source_index ON edges (target, source)")

        # Create the nodes table and its features
        cursor.execute("CREATE TABLE IF NOT EXISTS nodes (node_id SERIAL PRIMARY KEY)")
        
        # Dynamically add features if they do not exist
        cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name='nodes'")
        columns = [column[0] for column in cursor.fetchall()]
        
        for i in range(100):  # Assuming 100 features
            feature_name = f"feature_{i}"
            if feature_name not in columns:
                cursor.execute(f"ALTER TABLE nodes ADD COLUMN {feature_name} FLOAT")

        # Create the labels table and its index
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS labels (
                node_id INT REFERENCES nodes(node_id)
            )
        """)
        create_index("CREATE INDEX IF NOT EXISTS node_id_index ON labels (node_id)")

        connection.commit()
        cursor.close()

        # Function to load edges data into PostgreSQL
    
    def load_edges(connection, file_path, batch_size=50000):
        """
        Loads edges from a CSV file into the edges table in the database.

        Parameters:
        connection (psycopg2.extensions.connection): The database connection.
        edge_file (str): The path to the CSV file containing edge data.
        """
        print("Edges are loading!")
        cursor = connection.cursor()
        
        # Read all rows from the CSV file
        with open(file_path, 'r') as edges_file:
            edges_reader = csv.reader(edges_file)
            next(edges_reader)  # Skip header

            batch = []
            total_execution_time = 0.0

            for row in edges_reader:
                batch.append((int(row[0]), int(row[1])))

                if len(batch) >= batch_size:
                    # Process the batch
                    args_str = ','.join(cursor.mogrify("(%s,%s)", row).decode("utf-8") for row in batch)
                    query = f"INSERT INTO edges (source, target) VALUES {args_str};"
                    cursor.execute(f"EXPLAIN ANALYZE {query}")
                    explain_output = cursor.fetchall()

                    batch = []  # Clear batch

            # Insert remaining rows
            if batch:
                args_str = ','.join(cursor.mogrify("(%s,%s)", row).decode("utf-8") for row in batch)
                query = f"INSERT INTO edges (source, target) VALUES {args_str};"

            # Execute the query
            cursor.execute(query)

        connection.commit()
        cursor.close()
        print("Edge loading completed!")

    # Function to load nodes data into PostgreSQL 
    def load_nodes(connection, file_path, batch_size=50000):
        """
        Loads nodes from a CSV file into the nodes table in the database.

        Parameters:
        connection (psycopg2.extensions.connection): The database connection.
        node_file (str): The path to the CSV file containing node data.
        """
        print("Nodes are loading!")
        cursor = connection.cursor()
        
        # Read all rows from the CSV file
        with open(file_path, 'r') as nodes_file:
            nodes_reader = csv.reader(nodes_file)
            next(nodes_reader)  # Skip header

            batch = []
            total_execution_time = 0.0

            for row in nodes_reader:
                batch.append(tuple(map(float, row)))

                if len(batch) >= batch_size:
                    # Process the batch
                    args_str = ','.join(
                        cursor.mogrify("(" + ",".join(["%s"] * 100) + ")", row).decode("utf-8")
                        for row in batch
                    )
                    query = f"INSERT INTO nodes ({', '.join([f'feature_{i}' for i in range(100)])}) VALUES {args_str};"
                    cursor.execute(f"EXPLAIN ANALYZE {query}")
                    explain_output = cursor.fetchall()

                    batch = []  # Clear batch

            # Insert remaining rows
            if batch:
                args_str = ','.join(
                    cursor.mogrify("(" + ",".join(["%s"] * 100) + ")", row).decode("utf-8")
                    for row in batch
                )
                query = f"INSERT INTO nodes ({', '.join([f'feature_{i}' for i in range(100)])}) VALUES {args_str};"

            # Execute the query
            cursor.execute(query)

        connection.commit()
        cursor.close()
        print("Node loading completed!")

    # Function to load labels data into PostgreSQL
    def load_labels(connection, file_path, batch_size=50000):
        """
        Loads labels from a CSV file into the labels table in the database.

        Parameters:
        connection (psycopg2.extensions.connection): The database connection.
        label_file (str): The path to the CSV file containing label data.
        """
        print("Labels are loading!")
        cursor = connection.cursor()

        # Ensure the 'label' column exists
        cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name='labels'")
        existing_columns = set([column[0] for column in cursor.fetchall()])
        if "label" not in existing_columns:
            cursor.execute("ALTER TABLE labels ADD COLUMN label TEXT")
        
        # Read all rows from the CSV file
        with open(file_path, 'r') as labels_file:
            labels_reader = csv.reader(labels_file)
            next(labels_reader)  # Skip header

            batch = []
            total_execution_time = 0.0

            for idx, row in enumerate(labels_reader, start=1):
                batch.append((idx, row[0]))

                if len(batch) >= batch_size:
                    # Process the batch
                    args_str = ','.join(cursor.mogrify("(%s,%s)", row).decode("utf-8")
                                        for row in batch)
                    query = f"INSERT INTO labels (node_id, label) VALUES {args_str};"
                    cursor.execute(f"EXPLAIN ANALYZE {query}")
                    explain_output = cursor.fetchall()

                    batch = []  # Clear batch

            # Insert remaining rows
            if batch:
                args_str = ','.join(cursor.mogrify("(%s,%s)", row).decode("utf-8")
                                    for row in batch)
                query = f"INSERT INTO labels (node_id, label) VALUES {args_str};"

            # Execute the query
            cursor.execute(query)

        connection.commit()
        cursor.close()
        print("Label loading completed!")
        
    # Function to get the node table using PostgreSQL    
    def get_node_table_from_sql(dataset_name) -> np.array:  
        """
        Connects to the database, retrieves node features (excluding node_id) from the nodes table, 
        and returns them as a numpy array.
        
        Parameters:
        dataset_name (str): The name of the dataset (i.e., the database name).
        
        Returns:
        numpy.ndarray: A 2D numpy array of node features.
        """
        # Connect to the PostgreSQL database
        connector = sql_connector(dataset_name)
        
        # Create a cursor from the connection
        cursor = connector.cursor()
        
        # Execute the SQL query
        cursor.execute("SELECT * FROM nodes")  # Select all columns
        node_data = cursor.fetchall()  # Fetch all the rows returned by the query
        
        # Close the cursor
        cursor.close()

        # Convert the data to a numpy array
        node_data = np.array(node_data)

        # Debugging: Print the shape of the node_data
        print("Fetched node data shape:", node_data.shape)

        # Ensure the node data contains the correct number of feature columns
        if node_data.size == 0:  # Check if no data is returned
            raise ValueError("No data returned from the nodes table.")

        if node_data.shape[1] != 101:  # 100 features + 1 node_id column
            raise ValueError(f"Unexpected data shape. Ensure node table contains exactly 100 feature columns plus the node_id. Current shape: {node_data.shape}.")

        # Extract features excluding the node_id column
        node_features = node_data[:, 1:]  # This gets all columns except the first one (node_id)

        return node_features  # Return only the features

    # Function to get the edge index table using PostgreSQL
    def get_edge_table_from_sql(connection):
        """
        Retrieves the edge table from the database.

        Parameters:
        connection (psycopg2.extensions.connection): The database connection.

        Returns:
        list: List of edges as tuples.
        """
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM edges")
        edge_data = cursor.fetchall()

        if not edge_data:
            raise ValueError("No data returned from the edges table.")

        return edge_data

    #  Main block
    if __name__ == "__main__":
        """
        Main execution block of the script.

        This block initializes the parameters, creates the database and tables,
        loads data from CSV files, measures performance for various operations,
        and logs the results.

        The following operations are performed:
        - Loading edges, nodes, and labels.
        - Reading data with a specified number of hops.
        - Updating node features and edge weights.
        - Deleting all data.
        """

        # Define file paths 
        node_file = f'E:/Master Thesis/Thesis_Code/data/X_{nodes_count}_nodes_{edges_count}_edges.csv'
        edge_file = f'E:/Master Thesis/Thesis_Code/data/edge_index_{nodes_count}_nodes_{edges_count}_edges.csv'
        label_file = f'E:/Master Thesis/Thesis_Code/data/y_{nodes_count}_nodes_{edges_count}_edges.csv'

        nodes_count, edges_count = extract_file_info(node_file)
        db_name = f"db_{nodes_count}_nodes_{edges_count}_edges"

        # Create a new database
        create_database(db_name)    

        # Connect to the newly created database
        connection = psycopg2.connect(
            dbname=db_name,
            user='postgres',
            password='Berlin!321',
            host='localhost'
        )
        
        # Create tables with indexing
        create_tables_with_index(connection)

        # Load nodes, edges, and labels
        load_nodes(connection, node_file)
        load_edges(connection, edge_file)
        load_labels(connection, label_file)
    
        # Get node features and edge table
        node_features = get_node_table_from_sql(connection)
        print("Node Features Shape:", node_features.shape)

        # Load the labels from the labels table if needed
        cursor = connection.cursor()
        cursor.execute("SELECT label FROM labels")
        labels_data = cursor.fetchall()

        # Extract labels from fetched data
        node_labels = np.array([int(label[0]) for label in labels_data])  # Use float() if needed
        print("Node Labels Shape:", node_labels.shape)

        edge_data = get_edge_table_from_sql(connection)
        edge_index = np.array(edge_data, dtype=np.int64).T  # Ensure edge_index is of type int64
        print("Edge Data Length:", len(edge_index))

        # Close the connection
        connection.close()

        
        return node_features, node_labels, edge_index
    

In [13]:
def get_ppi_graph():
    
    # Imprort required libraries
    import psycopg2
    import numpy as np
    import csv
    import re
    import time
    from psycopg2 import sql

    # Helper function to connect to SQL database
    def sql_connector(dataset_name):
        connection = psycopg2.connect(
            dbname=db_name,
            user='postgres',
            password='Berlin!321',
            host='localhost'
        )

        return connection
       
    # Function to create a database
    def create_database(db_name):
        """
        Creates a new PostgreSQL database.

        Parameters:
        db_name (str): The name of the database to be created.
        """

        connection = psycopg2.connect(
            dbname='postgres',  # Connect to default db to create new db
            user='postgres',
            password='Berlin!321',
            host='localhost'
        )
        connection.autocommit = True

        cursor = connection.cursor()

        try:
            # Create the database
            cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name)))
            print(f"Database {db_name} created successfully.")
            
            # Connect to the newly created database and enable pg_stat_statements
            new_db_connection = psycopg2.connect(
                dbname=db_name,
                user='postgres',
                password='Berlin!321',
                host='localhost'
            )
            new_db_cursor = new_db_connection.cursor()
            
            new_db_cursor.close()
            new_db_connection.close()

        except psycopg2.Error as e:
            print(f"Error: {e}")
        finally:
            cursor.close()
            connection.close()

    # Function to create tables in the PostgreSQL database with indexing and track index creation times
    def create_tables_with_index(connection):
        """
        Creates the tables 'nodes', 'edges', and 'labels' in the connected PostgreSQL database, if they do not exist.
        Also adds indexes on certain columns to improve query performance.

        Parameters:
            connection (PostgreSQL connection object): Connection to the PostgreSQL database.

        Returns:
            tuple: A tuple containing two values:
                - mean_data_loading_index_time (float): Mean execution time for creating the indexes.
                - std_data_loading_index_time (float): Standard deviation of the execution time for creating the indexes.
        """
        cursor = connection.cursor()

        # Function to execute an index creation and track the time
        def create_index_and_time(query):
            cursor.execute(query)     # Execute index creation
            connection.commit()       # Commit after each index creation
            
        # Create the edges table and its indexes
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS edges (
                source INT, 
                target INT
            )
        """)
        
        # Create indexes and track their creation times
        create_index_and_time("CREATE INDEX IF NOT EXISTS source_index ON edges (source)")
        create_index_and_time("CREATE INDEX IF NOT EXISTS target_index ON edges (target)")
        create_index_and_time("CREATE INDEX IF NOT EXISTS source_target_index ON edges (source, target)")
        create_index_and_time("CREATE INDEX IF NOT EXISTS target_source_index ON edges (target, source)")

        # Create the nodes table and its features
        cursor.execute("CREATE TABLE IF NOT EXISTS nodes (node_id SERIAL PRIMARY KEY)")
        
        # Dynamically add features if they do not exist
        cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name='nodes'")
        columns = [column[0] for column in cursor.fetchall()]
        
        for i in range(100):  # Assuming 100 features
            feature_name = f"feature_{i}"
            if feature_name not in columns:
                cursor.execute(f"ALTER TABLE nodes ADD COLUMN {feature_name} FLOAT")

        # Create the labels table and its index
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS labels (
                node_id INT REFERENCES nodes(node_id)
            )
        """)
        create_index_and_time("CREATE INDEX IF NOT EXISTS node_id_index ON labels (node_id)")

        connection.commit()
        cursor.close()

    # Function to load edges data into PostgreSQL in batches
    def load_edges(connection, file_path, batch_size=30000):
        """
        Loads edges from a CSV file into the edges table in the database.

        Parameters:
        connection (psycopg2.extensions.connection): The database connection.
        edge_file (str): The path to the CSV file containing edge data.

        Returns:
        int: The number of edges loaded into the database.
        """

        print("Edges are loading!")
        cursor = connection.cursor()
        
        with open(file_path, 'r') as edges_file:
            edges_reader = csv.reader(edges_file)
            next(edges_reader)  # Skip header

            batch = []

            for row in edges_reader:
                batch.append((int(row[0]), int(row[1])))

                if len(batch) >= batch_size:
                    # Process the batch
                    args_str = ','.join(cursor.mogrify("(%s,%s)", row).decode("utf-8") for row in batch)
                    query = f"INSERT INTO edges (source, target) VALUES {args_str};"
                    cursor.execute(f"EXPLAIN ANALYZE {query}")
                    explain_output = cursor.fetchall()

                    batch = []  # Clear batch

            # Insert remaining rows
            if batch:
                args_str = ','.join(cursor.mogrify("(%s,%s)", row).decode("utf-8") for row in batch)
                query = f"INSERT INTO edges (source, target) VALUES {args_str};"
                cursor.execute(f"EXPLAIN ANALYZE {query}")
                explain_output = cursor.fetchall()

        connection.commit()
        cursor.close()
        print("Edge loading completed!")

    # Function to load nodes data into PostgreSQL in batches
    def load_nodes(connection, file_path, batch_size=30000):
        """
        Loads nodes from a CSV file into the nodes table in the database.

        Parameters:
        connection (psycopg2.extensions.connection): The database connection.
        node_file (str): The path to the CSV file containing node data.

        Returns:
        int: The number of nodes loaded into the database.
        """

        print("Nodes are loading!")
        cursor = connection.cursor()
        
        with open(file_path, 'r') as nodes_file:
            nodes_reader = csv.reader(nodes_file)
            next(nodes_reader)  # Skip header

            batch = []

            for row in nodes_reader:
                batch.append(tuple(map(float, row)))

                if len(batch) >= batch_size:
                    # Process the batch
                    args_str = ','.join(
                        cursor.mogrify("(" + ",".join(["%s"] * 50) + ")", row).decode("utf-8")
                        for row in batch
                    )
                    query = f"INSERT INTO nodes ({', '.join([f'feature_{i}' for i in range(50)])}) VALUES {args_str};"
                    cursor.execute(f"EXPLAIN ANALYZE {query}")
                    explain_output = cursor.fetchall()

                    batch = []  # Clear batch

            # Insert remaining rows
            if batch:
                args_str = ','.join(
                    cursor.mogrify("(" + ",".join(["%s"] * 50) + ")", row).decode("utf-8")
                    for row in batch
                )
                query = f"INSERT INTO nodes ({', '.join([f'feature_{i}' for i in range(50)])}) VALUES {args_str};"
                cursor.execute(f"EXPLAIN ANALYZE {query}")
                explain_output = cursor.fetchall()

        connection.commit()
        cursor.close()
        print("Node loading completed!")

    # Function to load labels data into PostgreSQL in batches
    def load_labels(connection, file_path, batch_size=30000):
        """
        Loads labels from a CSV file into the labels table in the database.

        Parameters:
        connection (psycopg2.extensions.connection): The database connection.
        label_file (str): The path to the CSV file containing label data.

        Returns:
        int: The number of labels loaded into the database.
        """

        print("Labels are loading!")
        cursor = connection.cursor()

        with open(file_path, 'r') as labels_file:
            labels_reader = csv.reader(labels_file)
            headers = next(labels_reader)  # Get headers from the file (all label columns)
            num_labels = len(headers)  # Number of labels is equal to the number of columns

            # Ensure the label columns exist and create them if they don't
            existing_columns = set()
            cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name='labels'")
            existing_columns.update([column[0] for column in cursor.fetchall()])

            for i in range(1, num_labels + 1):
                label_column = f"label_{i}"
                if label_column not in existing_columns:
                    cursor.execute(f"ALTER TABLE labels ADD COLUMN {label_column} TEXT")

            # Prepare SQL query for batch insertion
            columns = ",".join([f"label_{i}" for i in range(1, num_labels + 1)])
            query_template = f"INSERT INTO labels (node_id, {columns}) VALUES %s"
            
            batch = []

            for idx, row in enumerate(labels_reader, start=1):
                node_id = idx  # Use the row index as the node_id
                label_data = tuple(row)  # All columns are the label data
                batch.append((node_id, *label_data))
                
                if len(batch) >= batch_size:
                    # Process the batch
                    args_str = ','.join(cursor.mogrify(f"(%s, {','.join(['%s']*num_labels)})", row).decode("utf-8") 
                                        for row in batch)
                    query = query_template % args_str
                    cursor.execute(f"EXPLAIN ANALYZE {query}")
                    explain_output = cursor.fetchall()
                    
                    batch = []  # Clear batch

            # Insert any remaining rows
            if batch:
                args_str = ','.join(cursor.mogrify(f"(%s, {','.join(['%s']*num_labels)})", row).decode("utf-8") 
                                    for row in batch)
                query = query_template % args_str
                cursor.execute(f"EXPLAIN ANALYZE {query}")
                explain_output = cursor.fetchall()


        connection.commit()
        cursor.close()
        print("Label loading completed!")


    # Function to get the node table from PostgreSQL
    def get_node_table_from_sql(dataset_name) -> np.array:
        """
        Connects to the database, retrieves node features (excluding node_id) from the nodes table, 
        and returns them as a numpy array.
        
        Parameters:
        dataset_name (str): The name of the dataset (i.e., the database name).
        
        Returns:
        numpy.ndarray: A 2D numpy array of node features.
        """
        # Connect to the PostgreSQL database
        connector = sql_connector(dataset_name)
        
        # Create a cursor from the connection
        cursor = connector.cursor()
        
        # Execute the SQL query
        cursor.execute("SELECT * FROM nodes")  # Select all columns
        node_data = cursor.fetchall()  # Fetch all the rows returned by the query
        
        # Close the cursor
        cursor.close()

        # Convert the data to a numpy array
        node_data = np.array(node_data, dtype=np.float32)  # Ensure dtype is float32

        # Debugging: Print the shape of the node_data
        print("Fetched node data shape:", node_data.shape)

        # Ensure the node data contains the correct number of feature columns
        if node_data.size == 0:  # Check if no data is returned
            raise ValueError("No data returned from the nodes table.")

        if node_data.shape[1] != 101:  # 100 features + 1 node_id column
            raise ValueError(f"Unexpected data shape. Ensure node table contains exactly 100 feature columns plus the node_id. Current shape: {node_data.shape}.")

        # Extract features excluding the node_id column
        node_features = node_data[:, 1:]  # This gets all columns except the first one (node_id)

        return node_features.astype(np.float32)  # Ensure it's returned as float32
    
        
    # Function to get the edge index table from PostgreSQL
    def get_edge_table_from_sql(connection):
        """
        Retrieves the edge table from the database.

        Parameters:
        connection (psycopg2.extensions.connection): The database connection.

        Returns:
        np.array: A numpy array of edges as tuples of integers.
        """
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM edges")
        edge_data = cursor.fetchall()

        if not edge_data:
            raise ValueError("No data returned from the edges table.")

        # Convert edge data to numpy array and ensure it's dtype int64
        edge_data = np.array(edge_data, dtype=np.int64)

        return edge_data.T  # Return in transposed form (2 x num_edges)


    #  Main block
    if __name__ == "__main__":
        """
        Main execution block of the script.

        This block initializes the parameters, creates the database and tables,
        loads data from CSV files, measures performance for various operations,
        and logs the results.

        The following operations are performed:
        - Loading edges, nodes, and labels.
        - Reading data with a specified number of hops.
        - Updating node features and edge weights.
        - Deleting all data.
        """

        # Define file paths and database    
        edge_file = 'E:/Master Thesis/Thesis_Code/data/PPI_edge_index.csv'
        node_file = 'E:/Master Thesis/Thesis_Code/data/PPI_X.csv'
        label_file = 'E:/Master Thesis/Thesis_Code/data/PPI_y.csv'  

        db_name = f"query_ops_PPI"
        dataset_name = f"dataset_PPI"

        # Create a new database
        create_database(db_name)    

        # Connect to the newly created database
        connection = psycopg2.connect(
            dbname=db_name,
            user='postgres',
            password='Berlin!321',
            host='localhost'
        )
        
        # Create tables with indexing
        create_tables_with_index(connection)

        # Load nodes, edges, and labels
        load_nodes(connection, node_file)
        load_edges(connection, edge_file)
        load_labels(connection, label_file)

        # Get node features and edge table
        node_features = get_node_table_from_sql(connection)
        print("Node Features Shape:", node_features.shape)

        # # Load the labels from the labels table if needed
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM labels")
        labels_data = cursor.fetchall()

        # Extract labels from fetched data
        # node_labels = np.array([label[0] for label in labels_data])  # Assuming label is in the first column
        node_labels = np.array([row[1:] for row in labels_data], dtype=np.float32)  
        print("Node Labels Shape:", node_labels.shape)
        
        edge_index = get_edge_table_from_sql(connection)
        print("Edge Data Length:", len(edge_index))

        # Close the connection
        connection.close()

        return node_features, node_labels, edge_index

        


In [4]:
# Define node and edge counts for the dataset
nodes_count = 1000  # Change this as needed
edges_count = 5

In [16]:
# Generate the graph data using get_graph
node_features, node_labels, edge_index = get_graph(nodes_count, edges_count)
# node_features, node_labels, edge_index = get_graph_batches(nodes_count, edges_count)
# node_features, node_labels, edge_index = get_ppi_graph()

# # Print the types and shapes to inspect the arrays
# print(f"Node Features: {type(node_features)}, dtype: {node_features.dtype}, shape: {node_features.shape}")
# print(f"Node Labels: {type(node_labels)}, dtype: {node_labels.dtype}, shape: {node_labels.shape}")
# print(f"Edge Index: {type(edge_index)}, dtype: {edge_index.dtype}, shape: {edge_index.shape}")


data = Data(x = torch.from_numpy(node_features).type(torch.float32), y = torch.from_numpy(node_labels).type(torch.float32), edge_index = torch.from_numpy(edge_index).type(torch.long))
data        

Database query_ops_PPI created successfully.
Nodes are loading!
Node loading completed!
Edges are loading!
Edge loading completed!
Labels are loading!
Label loading completed!
Fetched node data shape: (44906, 101)
Node Features Shape: (44906, 100)
Node Labels Shape: (44906, 121)
Edge Data Length: 2


Data(x=[44906, 100], edge_index=[2, 1226368], y=[44906, 121])

In [17]:
#  Model

import torch
from torch_geometric.nn.conv import GATConv
from memory_profiler import profile

class GNN(torch.nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, heads = 8):
        super(GNN, self).__init__()
        self.in_gat = GATConv(in_dim, hidden_dim, heads = heads, concat=True)
        self.hidden_gat = GATConv(hidden_dim*8, hidden_dim, heads = heads, concat=True)
        self.out_gat = GATConv(hidden_dim*8, out_dim, heads = heads, concat=False)

    @profile
    def forward(self,graph):
        x, edge_index = graph.x, graph.edge_index
        x = torch.relu(self.in_gat(x, edge_index))
        x = torch.relu(self.hidden_gat(x, edge_index))
        x = self.out_gat(x, edge_index)
        return x

model = GNN(100, 256, 1)
model.train()

GNN(
  (in_gat): GATConv(100, 256, heads=8)
  (hidden_gat): GATConv(2048, 256, heads=8)
  (out_gat): GATConv(2048, 1, heads=8)
)

In [18]:
# from torch_geometric.data import Data

# data = Data(x = x, edge_index= edge_index)
from memory_profiler import memory_usage

def run_forward_pass(model, input_data):
    return model(input_data)

mem_usage = memory_usage(proc=(run_forward_pass, (model, data), {}))
print(f"Peak memory usage: {max(mem_usage)} MiB")

ERROR: Could not find file C:\Users\Dell\AppData\Local\Temp/ipykernel_8924/2240319332.py


RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 10414276608 bytes.